In [ ]:
# import arcpy, os, math, datetime as dt
# from arcpy.sa import *

# # --- CONFIG ---
# GDB        = r"C:\Users\colto\Documents\tw_project\tw_project\tw_project.gdb"
# POINTS_FC  = r"C:\Users\colto\Documents\tw_project\tw_project\tw_project.gdb\cities_cum"  # FC inside GDB (no .shp)
# TIME_FIELD = "time_bin_Converted"   # must be Date type
# BIN_HOURS  = 4
# CELL_SIZE  = 1000      # meters
# RADIUS     = 18000     # meters
# MD_NAME    = "KD_Time"
# PREFIX     = "kd_"

# arcpy.CheckOutExtension("Spatial")
# arcpy.env.overwriteOutput = True
# arcpy.env.workspace = GDB
# arcpy.env.scratchWorkspace = GDB  # avoid temp path weirdness

# # --- 0) Ensure projected to METERS (5070). Use GDB target, not in_memory. ---
# sr_in = arcpy.Describe(POINTS_FC).spatialReference
# if sr_in.type == "Geographic":
#     PTS_METERS = os.path.join(GDB, "cities_cum_5070")
#     if arcpy.Exists(PTS_METERS):
#         arcpy.management.Delete(PTS_METERS)
#     arcpy.management.Project(POINTS_FC, PTS_METERS, arcpy.SpatialReference(5070))  # NAD83 / Conus Albers
#     if not arcpy.Exists(PTS_METERS):
#         raise RuntimeError("Project() failed:\n" + arcpy.GetMessages(2))
# else:
#     PTS_METERS = POINTS_FC

# # Now safe to Describe
# sr = arcpy.Describe(PTS_METERS).spatialReference
# arcpy.env.outputCoordinateSystem = sr
# arcpy.env.extent = arcpy.Describe(PTS_METERS).extent
# arcpy.env.cellSize = CELL_SIZE

# # --- 1) Collect time range ---
# def _iter_times(fc, fld):
#     with arcpy.da.SearchCursor(fc, [fld]) as cur:
#         for (t,) in cur:
#             if t: yield t

# times = list(_iter_times(PTS_METERS, TIME_FIELD))
# if not times:
#     raise RuntimeError(f"No valid times in field '{TIME_FIELD}'.")
# tmin, tmax = min(times), max(times)
# start = dt.datetime(tmin.year, tmin.month, tmin.day, (tmin.hour // BIN_HOURS) * BIN_HOURS)

# # --- 2) Loop bins → KDE rasters into GDB ---
# created = []
# cur = start
# while cur <= tmax:
#     nxt = cur + dt.timedelta(hours=BIN_HOURS)
#     where = (f"{arcpy.AddFieldDelimiters(PTS_METERS, TIME_FIELD)} >= TIMESTAMP '{cur:%Y-%m-%d %H:%M:%S}' "
#              f"AND {arcpy.AddFieldDelimiters(PTS_METERS, TIME_FIELD)} < TIMESTAMP '{nxt:%Y-%m-%d %H:%M:%S}'")
#     lyr_name = f"lyr_{cur:%Y%m%d_%H%M}"
#     lyr = arcpy.management.MakeFeatureLayer(PTS_METERS, lyr_name, where)
#     try:
#         if int(arcpy.management.GetCount(lyr).getOutput(0)) > 0:
#             out_name = f"{PREFIX}{cur:%Y%m%d_%H%M}"
#             out_path = os.path.join(GDB, out_name)
#             if arcpy.Exists(out_path): arcpy.management.Delete(out_path)
#             kd = KernelDensity(lyr, population_field=None,
#                                cell_size=CELL_SIZE, search_radius=RADIUS,
#                                out_cell_values="DENSITIES", method="PLANAR")
#             kd.save(out_path)
#             created.append(out_name)
#     except Exception:
#         raise RuntimeError("KernelDensity failed:\n" + arcpy.GetMessages(2))
#     finally:
#         arcpy.management.Delete(lyr)
#     cur = nxt

# if not created:
#     raise RuntimeError("No rasters created (empty bins or bad time field).")

# # --- 3) Time-enabled Mosaic Dataset ---
# if arcpy.Exists(os.path.join(GDB, MD_NAME)):
#     arcpy.management.Delete(os.path.join(GDB, MD_NAME))

# MD = arcpy.management.CreateMosaicDataset(
#     GDB, MD_NAME, sr, pixel_type="32_BIT_FLOAT"
# ).getOutput(0)

# # Add only the rasters we just created (one call per raster path)
# for nm in created:
#     ras_path = os.path.join(GDB, nm)
#     arcpy.management.AddRastersToMosaicDataset(
#         in_mosaic_dataset=MD,
#         raster_type="Raster Dataset",
#         input_path=ras_path,
#         update_cellsize_ranges="UPDATE_CELL_SIZES",
#         update_boundary="UPDATE_BOUNDARY",
#         update_overviews="NO_OVERVIEWS"
#     )


# # Remove any items not in our created set
# keep = set(created)
# to_delete = []
# with arcpy.da.SearchCursor(MD, ["OID@", "Name"]) as rows:
#     for oid, nm in rows:
#         if nm not in keep:
#             to_delete.append(oid)
# if to_delete:
#     q = "OBJECTID IN ({})".format(",".join(map(str, to_delete)))
#     arcpy.management.DeleteRows(arcpy.management.MakeTableView(MD, "md_del", q))

# # Add StartTime and populate from Name 'kd_YYYYMMDD_HHMM'
# if "StartTime" not in [f.name for f in arcpy.ListFields(MD)]:
#     arcpy.management.AddField(MD, "StartTime", "DATE")
# code_block = """import datetime
# def parse_name(nm):
#     return datetime.datetime.strptime(nm.split('_',1)[-1], '%Y%m%d_%H%M')
# """
# arcpy.management.CalculateField(MD, "StartTime", "parse_name(!Name!)", "PYTHON3", code_block)
# # arcpy.management.EnableTime(MD, "StartTime", "Single", timeStepInterval=str(BIN_HOURS), timeStepUnits="HOURS")

# # Build pyramids & stats (correct function name/case)
# arcpy.management.BuildPyramidsandStatistics(MD, skip_existing="OVERWRITE")

# print(f"OK: {len(created)} KDE slices → {MD}")


In [ ]:
# Version 2

In [ ]:
# import arcpy, os, math, datetime as dt
# from arcpy.sa import *

# # ---------------- CONFIG ----------------
# GDB         = r"C:\Users\colto\Documents\tw_project\tw_project\tw_project.gdb"
# POINTS_FC   = r"C:\Users\colto\Documents\tw_project\tw_project\tw_project.gdb\cities_cum"  # FC in GDB (no .shp)
# TIME_FIELD  = "time_bin_Converted"   # must be Date type
# BIN_HOURS   = 4

# # Kernel Density params (METERS)
# CELL_SIZE_M = 1000   # 1 km cells (Figure-5 look)
# RADIUS_M    = 18000    # 18 km bandwidth

# MD_NAME     = "KD_Time"            # mosaic dataset name
# PREFIX      = "kd_"                # slice name prefix kd_YYYYMMDD_HHMM

# # Optional: If you have an AOI polygon, uncomment to limit processing window (faster/safer)
# AOI = None  # r"C:\path\to\your\gdb\aoi_polygon"

# # -------------- SETUP -------------------
# arcpy.CheckOutExtension("Spatial")
# arcpy.env.overwriteOutput = True
# arcpy.env.workspace = GDB
# arcpy.env.scratchWorkspace = GDB

# # 0) Project points to a meters-based CRS (EPSG:5070) for true meter units
# sr_in = arcpy.Describe(POINTS_FC).spatialReference
# PTS_METERS = os.path.join(GDB, "cities_cum_5070")
# if not arcpy.Exists(PTS_METERS):
#     if sr_in.type == "Geographic" or sr_in.linearUnitName.lower() in ("", "degree", "degrees"):
#         arcpy.management.Project(POINTS_FC, PTS_METERS, arcpy.SpatialReference(5070))  # NAD83 / Conus Albers
#     else:
#         # Already projected — make a clean copy with a known name
#         arcpy.management.CopyFeatures(POINTS_FC, PTS_METERS)

# # Set processing envs to projected space
# sr = arcpy.Describe(PTS_METERS).spatialReference
# arcpy.env.outputCoordinateSystem = sr
# arcpy.env.cellSize = CELL_SIZE_M
# if AOI:
#     arcpy.env.mask = AOI
#     arcpy.env.extent = arcpy.Describe(AOI).extent

# # 1) Determine time range
# def iter_times(fc, fld):
#     with arcpy.da.SearchCursor(fc, [fld]) as rows:
#         for (t,) in rows:
#             if t:
#                 yield t

# times = list(iter_times(PTS_METERS, TIME_FIELD))
# if not times:
#     raise RuntimeError(f"No valid times in field '{TIME_FIELD}'.")

# tmin, tmax = min(times), max(times)
# start = dt.datetime(tmin.year, tmin.month, tmin.day, (tmin.hour // BIN_HOURS) * BIN_HOURS)

# # 2) Per-bin Kernel Density (1 km cell, 18 km radius) saved to the GDB
# created = []
# cur = start
# while cur <= tmax:
#     nxt = cur + dt.timedelta(hours=BIN_HOURS)
#     where = (f"{arcpy.AddFieldDelimiters(PTS_METERS, TIME_FIELD)} >= TIMESTAMP '{cur:%Y-%m-%d %H:%M:%S}' "
#              f"AND {arcpy.AddFieldDelimiters(PTS_METERS, TIME_FIELD)} < TIMESTAMP '{nxt:%Y-%m-%d %H:%M:%S}'")

#     lyr_name = f"bin_{cur:%Y%m%d_%H%M}"
#     arcpy.management.MakeFeatureLayer(PTS_METERS, lyr_name, where)
#     try:
#         if int(arcpy.management.GetCount(lyr_name).getOutput(0)) > 0:
#             print(cur, tmax)
#             out_name = f"{PREFIX}{cur:%Y%m%d_%H%M}"
#             out_path = os.path.join(GDB, out_name)
#             if arcpy.Exists(out_path):
#                 arcpy.management.Delete(out_path)

#             kd = KernelDensity(
#                 in_features=lyr_name,
#                 population_field=None,
#                 cell_size=CELL_SIZE_M,
#                 search_radius=RADIUS_M,
#                 out_cell_values="DENSITIES",
#                 method="PLANAR"
#             )
#             kd.save(out_path)
#             created.append(out_name)
#     except Exception:
#         raise RuntimeError("KernelDensity failed:\n" + arcpy.GetMessages(2))
#     finally:
#         arcpy.management.Delete(lyr_name)
#     cur = nxt

# if not created:
#     raise RuntimeError("No rasters created. Check time field values and bin size.")

# # 3) Create a time-aware Mosaic Dataset and add the rasters we just made
# if arcpy.Exists(os.path.join(GDB, MD_NAME)):
#     arcpy.management.Delete(os.path.join(GDB, MD_NAME))

# MD = arcpy.management.CreateMosaicDataset(
#     in_workspace=GDB,
#     in_mosaicdataset_name=MD_NAME,
#     coordinate_system=sr,
#     pixel_type="32_BIT_FLOAT"
# ).getOutput(0)

# # Add each slice explicitly (robust for FGDB rasters)
# for nm in created:
#     print(nm)
#     arcpy.management.AddRastersToMosaicDataset(
#         in_mosaic_dataset=MD,
#         raster_type="Raster Dataset",
#         input_path=os.path.join(GDB, nm),
#         update_cellsize_ranges="UPDATE_CELL_SIZES",
#         update_boundary="UPDATE_BOUNDARY",
#         update_overviews="NO_OVERVIEWS"
#     )

# # 4) Add StartTime on the mosaic items and populate from item Name (kd_YYYYMMDD_HHMM)
# if "StartTime" not in [f.name for f in arcpy.ListFields(MD)]:
#     arcpy.management.AddField(MD, "StartTime", "DATE")

# code_block = """import datetime
# def parse_name(nm):
#     # expects 'kd_YYYYMMDD_HHMM'
#     return datetime.datetime.strptime(nm.split('_',1)[-1], '%Y%m%d_%H%M')
# """
# arcpy.management.CalculateField(MD, "StartTime", "parse_name(!Name!)", "PYTHON3", code_block)
# # arcpy.management.EnableTime(MD, "StartTime", "Single", timeStepInterval=str(BIN_HOURS), timeStepUnits="HOURS")

# # Build pyramids & stats (correct function name/case)
# arcpy.management.BuildPyramidsandStatistics(MD, skip_existing="OVERWRITE")

# print(f"OK: {len(created)} KDE slices → {MD}")


In [ ]:
#Version 3

In [ ]:
# import arcpy, os, math, datetime as dt
# from arcpy.sa import *

# # ---------------- CONFIG ----------------
# GDB         = r"C:\Users\colto\Documents\tw_project\tw_project\tw_project.gdb"
# POINTS_FC   = r"C:\Users\colto\Documents\tw_project\tw_project\tw_project.gdb\cities_inc_helene"  # FC in GDB (no .shp)
# TIME_FIELD  = "time_bin_Converted"   # must be Date type
# BIN_HOURS   = 4

# # Kernel Density params (METERS)
# CELL_SIZE_M = 1000   # 1 km cells (Figure-5 look)
# RADIUS_M    = 18000    # 18 km bandwidth

# MD_NAME     = "KD_Time_cities_inc_helene"            # mosaic dataset name
# PREFIX      = "kd_cities_inc_helene_"                # slice name prefix kd_YYYYMMDD_HHMM

# # Optional: If you have an AOI polygon, uncomment to limit processing window (faster/safer)
# AOI = None  # r"C:\path\to\your\gdb\aoi_polygon"

# # -------------- SETUP -------------------
# arcpy.CheckOutExtension("Spatial")
# arcpy.env.overwriteOutput = True
# arcpy.env.workspace = GDB
# arcpy.env.scratchWorkspace = GDB

# # 0) Project points to a meters-based CRS (EPSG:5070) for true meter units
# sr_in = arcpy.Describe(POINTS_FC).spatialReference
# PTS_METERS = os.path.join(GDB, "cities_cum_5070")
# if not arcpy.Exists(PTS_METERS):
#     if sr_in.type == "Geographic" or sr_in.linearUnitName.lower() in ("", "degree", "degrees"):
#         arcpy.management.Project(POINTS_FC, PTS_METERS, arcpy.SpatialReference(5070))  # NAD83 / Conus Albers
#     else:Z
#         # Already projected — make a clean copy with a known name
#         arcpy.management.CopyFeatures(POINTS_FC, PTS_METERS)

# # Set processing envs to projected space
# sr = arcpy.Describe(PTS_METERS).spatialReference
# arcpy.env.outputCoordinateSystem = sr
# arcpy.env.cellSize = CELL_SIZE_M
# if AOI:
#     arcpy.env.mask = AOI
#     arcpy.env.extent = arcpy.Describe(AOI).extent

# # --- Recreate Date field from time_bin text ---
# SRC_TEXT = "time_bin"            # the text field
# DST_DATE = "time_bin_Converted"  # new Date field

# # Add Date field if missing
# if DST_DATE not in [f.name for f in arcpy.ListFields(PTS_METERS)]:
#     arcpy.management.AddField(PTS_METERS, DST_DATE, "DATE")

# # Convert string to datetime
# code_block = r"""
# import datetime
# def to_dt(s):
#     try:
#         s = (s or '').strip().replace('T',' ')
#         return datetime.datetime.strptime(s, '%Y-%m-%d %H:%M:%S')
#     except Exception:
#         try:
#             return datetime.datetime.strptime(s, '%Y-%m-%d %H:%M')
#         except Exception:
#             return None
# """
# arcpy.management.CalculateField(
#     PTS_METERS, DST_DATE, f"to_dt(!{SRC_TEXT}!)", "PYTHON3", code_block
# )

# # Tell the script to use that Date field
# TIME_FIELD = DST_DATE
# print(f"✅ Created/verified {DST_DATE} on {PTS_METERS}")



# # 1) Determine time range
# def iter_times(fc, fld):
#     with arcpy.da.SearchCursor(fc, [fld]) as rows:
#         for (t,) in rows:
#             if t:
#                 yield t

# times = list(iter_times(PTS_METERS, TIME_FIELD))
# if not times:
#     raise RuntimeError(f"No valid times in field '{TIME_FIELD}'.")

# tmin, tmax = min(times), max(times)
# start = dt.datetime(tmin.year, tmin.month, tmin.day, (tmin.hour // BIN_HOURS) * BIN_HOURS)

# # 2) Per-bin Kernel Density (1 km cell, 18 km radius) saved to the GDB
# created = []
# cur = start
# while cur <= tmax:
#     nxt = cur + dt.timedelta(hours=BIN_HOURS)
#     where = (f"{arcpy.AddFieldDelimiters(PTS_METERS, TIME_FIELD)} >= TIMESTAMP '{cur:%Y-%m-%d %H:%M:%S}' "
#              f"AND {arcpy.AddFieldDelimiters(PTS_METERS, TIME_FIELD)} < TIMESTAMP '{nxt:%Y-%m-%d %H:%M:%S}'")

#     lyr_name = f"bin_{cur:%Y%m%d_%H%M}"
#     arcpy.management.MakeFeatureLayer(PTS_METERS, lyr_name, where)
#     try:
#         if int(arcpy.management.GetCount(lyr_name).getOutput(0)) > 0:
#             print(cur, tmax)
#             out_name = f"{PREFIX}{cur:%Y%m%d_%H%M}"
#             out_path = os.path.join(GDB, out_name)
#             if arcpy.Exists(out_path):
#                 arcpy.management.Delete(out_path)

#             kd = KernelDensity(
#                 in_features=lyr_name,
#                 population_field=None,
#                 cell_size=CELL_SIZE_M,
#                 search_radius=RADIUS_M,z
#                 out_cell_values="DENSITIES",
#                 method="PLANAR"
#             )
#             kd.save(out_path)
#             created.append(out_name)
#     except Exception:
#         raise RuntimeError("KernelDensity failed:\n" + arcpy.GetMessages(2))
#     finally:
#         arcpy.management.Delete(lyr_name)
#     cur = nxt

# if not created:
#     raise RuntimeError("No rasters created. Check time field values and bin size.")

# # 3) Create a time-aware Mosaic Dataset and add the rasters we just made
# if arcpy.Exists(os.path.join(GDB, MD_NAME)):
#     arcpy.management.Delete(os.path.join(GDB, MD_NAME))

# MD = arcpy.management.CreateMosaicDataset(
#     in_workspace=GDB,
#     in_mosaicdataset_name=MD_NAME,
#     coordinate_system=sr,
#     pixel_type="32_BIT_FLOAT"
# ).getOutput(0)

# # Add each slice explicitly (robust for FGDB rasters)
# for nm in created:
#     print(nm)
#     arcpy.management.AddRastersToMosaicDataset(
#         in_mosaic_dataset=MD,
#         raster_type="Raster Dataset",
#         input_path=os.path.join(GDB, nm),
#         update_cellsize_ranges="UPDATE_CELL_SIZES",
#         update_boundary="UPDATE_BOUNDARY",
#         update_overviews="NO_OVERVIEWS"
#     )

# # 4) Add StartTime on the mosaic items and populate from item Name (kd_YYYYMMDD_HHMM)
# if "StartTime" not in [f.name for f in arcpy.ListFields(MD)]:
#     arcpy.management.AddField(MD, "StartTime", "DATE")

# code_block = """import datetime
# def parse_name(nm):
#     # expects 'kd_YYYYMMDD_HHMM' 
#     print(datetime.datetime.strptime(nm.[-13], '%Y%m%d_%H%M'))
#     return datetime.datetime.strptime(nm.split('_',1)[-1], '%Y%m%d_%H%M')
# """
# arcpy.management.CalculateField(MD, "StartTime", "parse_name(!Name!)", "PYTHON3", code_block)
# # arcpy.management.EnableTime(MD, "StartTime", "Single", timeStepInterval=str(BIN_HOURS), timeStepUnits="HOURS")

# # Build pyramids & stats (correct function name/case)
# arcpy.management.BuildPyramidsandStatistics(MD, skip_existing="OVERWRITE")

# print(f"OK: {len(created)} KDE slices → {MD}")


In [ ]:
#Version 4

In [ ]:
# import arcpy, os, re, datetime as dt
# from arcpy.sa import *

# # ---------------- CONFIG ----------------
# current_file = "cities_cum_helene"
# GDB         = r"C:\Users\colto\Documents\tw_project\tw_project\tw_project.gdb"
# POINTS_FC   = r"C:\Users\colto\Documents\tw_project\tw_project\tw_project.gdb\cities_cum_helene"
# BIN_HOURS   = 4

# CELL_SIZE_M = 1000
# RADIUS_M    = 18000
# MD_NAME     = f"""KD_Time_{current_file}_pop"""
# PREFIX      = f"""kd_{current_file}_pop_"""

# arcpy.CheckOutExtension("Spatial")
# arcpy.env.overwriteOutput  = True
# arcpy.env.workspace        = GDB
# arcpy.env.scratchWorkspace = GDB

# # --- 1) Convert string field -> Date field (official ArcGIS tool) ---
# SRC_FC   = POINTS_FC
# SRC_TEXT = "time_bin"                 # your text time field
# DST_DATE = "time_bin_Converted"       # name of new Date field
# FORMAT   = "yyyy-MM-dd HH:mm:ss"      # adjust if yours differs

# # Only run conversion if needed
# if DST_DATE not in [f.name for f in arcpy.ListFields(SRC_FC)]:
#     arcpy.management.ConvertTimeField(
#         in_table=SRC_FC,
#         input_time_field=SRC_TEXT,
#         input_time_format=FORMAT,
#         output_time_field=DST_DATE,
#         output_time_type="DATE"
#     )
#     print(f"✅ Created {DST_DATE} from {SRC_TEXT}")
# else:
#     print(f"✅ {DST_DATE} already exists")

# TIME_FIELD = DST_DATE

# # 0) Project points to a meters-based CRS (EPSG:5070) for true meter units
# sr_in = arcpy.Describe(POINTS_FC).spatialReference
# PTS_METERS = os.path.join(GDB, f"""{current_file}_5070""")
# if not arcpy.Exists(PTS_METERS):
#     if sr_in.type == "Geographic" or sr_in.linearUnitName.lower() in ("", "degree", "degrees"):
#         arcpy.management.Project(POINTS_FC, PTS_METERS, arcpy.SpatialReference(5070))  # NAD83 / Conus Albers
#     else:
#         # Already projected — make a clean copy with a known name
#         arcpy.management.CopyFeatures(POINTS_FC, PTS_METERS)

# # Set processing envs to projected space
# sr = arcpy.Describe(PTS_METERS).spatialReference
# arcpy.env.outputCoordinateSystem = sr
# arcpy.env.cellSize = CELL_SIZE_M
# if AOI:
#     arcpy.env.mask = AOI
#     arcpy.env.extent = arcpy.Describe(AOI).extent

# # 1) Determine time range
# def iter_times(fc, fld):
#     with arcpy.da.SearchCursor(fc, [fld]) as rows:
#         print(rows)
#         for (t,) in rows:
#             if t:
#                 yield t

# times = list(iter_times(PTS_METERS, TIME_FIELD))

# if not times:
#     raise RuntimeError(f"No valid times in field '{TIME_FIELD}'.")

# tmin, tmax = min(times), max(times)
# start = dt.datetime(tmin.year, tmin.month, tmin.day, (tmin.hour // BIN_HOURS) * BIN_HOURS)

# # 2) Per-bin Kernel Density (1 km cell, 18 km radius) saved to the GDB
# created = []
# cur = start
# while cur <= tmax:
#     nxt = cur + dt.timedelta(hours=BIN_HOURS)
#     where = (f"{arcpy.AddFieldDelimiters(PTS_METERS, TIME_FIELD)} >= TIMESTAMP '{cur:%Y-%m-%d %H:%M:%S}' "
#              f"AND {arcpy.AddFieldDelimiters(PTS_METERS, TIME_FIELD)} < TIMESTAMP '{nxt:%Y-%m-%d %H:%M:%S}'")

#     lyr_name = f"bin_{cur:%Y%m%d_%H%M}"
    
#     arcpy.management.MakeFeatureLayer(PTS_METERS, lyr_name, where)
#     for f in arcpy.ListFields(lyr_name):
#         if f.name == "population":
#             print(f"Field '{f.name}': type={f.type}")
#     print(foo)
#     try:
#         if int(arcpy.management.GetCount(lyr_name).getOutput(0)) > 0:
#             print(cur, tmax)
#             out_name = f"{PREFIX}{cur:%Y%m%d_%H%M}"
#             out_path = os.path.join(GDB, out_name)
#             if arcpy.Exists(out_path):
#                 arcpy.management.Delete(out_path)

#             kd = KernelDensity(
#                 in_features=lyr_name,
#                 population_field="population",
#                 cell_size=CELL_SIZE_M,
#                 search_radius=RADIUS_M,
#                 out_cell_values="DENSITIES",
#                 method="PLANAR"
#             )
#             kd.save(out_path)
#             created.append(out_name)
#     except Exception:
#         raise RuntimeError("KernelDensity failed:\n" + arcpy.GetMessages(2))
#     finally:
#         arcpy.management.Delete(lyr_name)
#     cur = nxt

# if not created:
#     raise RuntimeError("No rasters created. Check time field values and bin size.")

# # 3) Create a time-aware Mosaic Dataset and add the rasters we just made
# if arcpy.Exists(os.path.join(GDB, MD_NAME)):
#     arcpy.management.Delete(os.path.join(GDB, MD_NAME))

# MD = arcpy.management.CreateMosaicDataset(
#     in_workspace=GDB,
#     in_mosaicdataset_name=MD_NAME,
#     coordinate_system=sr,
#     pixel_type="32_BIT_FLOAT"
# ).getOutput(0)

# # Add each slice explicitly (robust for FGDB rasters)
# for nm in created:
#     print(nm)
#     arcpy.management.AddRastersToMosaicDataset(
#         in_mosaic_dataset=MD,
#         raster_type="Raster Dataset","C:\\Users\\colto\\Documents\\GitHub\\Tweet_project\\arcgis_outputs\\helene\\cumulative\\cities_CUMULATIVE_ALL.shp"
#         input_path=os.path.join(GDB, nm),
#         update_cellsize_ranges="UPDATE_CELL_SIZES",
#         update_boundary="UPDATE_BOUNDARY",
#         update_overviews="NO_OVERVIEWS"
#     )

# # 4) Add StartTime on the mosaic items and populate from item Name (kd_YYYYMMDD_HHMM)
# if "StartTime" not in [f.name for f in arcpy.ListFields(MD)]:
#     arcpy.management.AddField(MD, "StartTime", "DATE")

# code_block = """import datetime
# def parse_name(nm):
#     # expects 'kd_YYYYMMDD_HHMM' 
#     return datetime.datetime.strptime(nm[-13:], '%Y%m%d_%H%M')
# """
# arcpy.management.CalculateField(MD, "StartTime", "parse_name(!Name!)", "PYTHON3", code_block)
# # arcpy.management.EnableTime(MD, "StartTime", "Single", timeStepInterval=str(BIN_HOURS), timeStepUnits="HOURS")

# # Build pyramids & stats (correct function name/case)
# arcpy.management.BuildPyramidsandStatistics(MD, skip_existing="OVERWRITE")

# print(f"OK: {len(created)} KDE slices → {MD}")


In [ ]:
#version 5

In [62]:
import arcpy, os, re, datetime as dt
from arcpy.sa import *

# ---------------- CONFIG ----------------
current_file = "cities_cum_helene"
GDB         = r"C:\Users\colto\Documents\tw_project\tw_project\tw_project.gdb"
POINTS_FC   = r"C:\Users\colto\Documents\tw_project\tw_project\tw_project.gdb\cities_cum_helene"
BIN_HOURS   = 4

CELL_SIZE_M = 1000
RADIUS_M    = 18000
MD_NAME     = f"""KD_Time_{current_file}_pop"""
PREFIX      = f"""kd_{current_file}_pop_"""

arcpy.CheckOutExtension("Spatial")
arcpy.env.overwriteOutput  = True
arcpy.env.workspace        = GDB
arcpy.env.scratchWorkspace = GDB

# --- 1) Convert string field -> Date field (official ArcGIS tool) ---
SRC_FC   = POINTS_FC
SRC_TEXT = "time_bin"                 # your text time field
DST_DATE = "time_bin_Converted"       # name of new Date field
FORMAT   = "yyyy-MM-dd HH:mm:ss"      # adjust if yours differs

# Only run conversion if needed
if DST_DATE not in [f.name for f in arcpy.ListFields(SRC_FC)]:
    arcpy.management.ConvertTimeField(
        in_table=SRC_FC,
        input_time_field=SRC_TEXT,
        input_time_format=FORMAT,
        output_time_field=DST_DATE,
        output_time_type="DATE"
    )
    print(f"✅ Created {DST_DATE} from {SRC_TEXT}")
else:
    print(f"✅ {DST_DATE} already exists")

TIME_FIELD = DST_DATE

# 0) Project points to a meters-based CRS (EPSG:5070) for true meter units
sr_in = arcpy.Describe(POINTS_FC).spatialReference
PTS_METERS = os.path.join(GDB, f"""{current_file}_5070""")
if not arcpy.Exists(PTS_METERS):
    if sr_in.type == "Geographic" or sr_in.linearUnitName.lower() in ("", "degree", "degrees"):
        arcpy.management.Project(POINTS_FC, PTS_METERS, arcpy.SpatialReference(5070))  # NAD83 / Conus Albers
    else:
        # Already projected — make a clean copy with a known name
        arcpy.management.CopyFeatures(POINTS_FC, PTS_METERS)

# Set processing envs to projected space
sr = arcpy.Describe(PTS_METERS).spatialReference
arcpy.env.outputCoordinateSystem = sr
arcpy.env.cellSize = CELL_SIZE_M
if AOI:
    arcpy.env.mask = AOI
    arcpy.env.extent = arcpy.Describe(AOI).extent

# 1) Determine time range
def iter_times(fc, fld):
    with arcpy.da.SearchCursor(fc, [fld]) as rows:
        print(rows)
        for (t,) in rows:
            if t:
                yield t

times = list(iter_times(PTS_METERS, TIME_FIELD))

if not times:
    raise RuntimeError(f"No valid times in field '{TIME_FIELD}'.")

tmin, tmax = min(times), max(times)
start = dt.datetime(tmin.year, tmin.month, tmin.day, (tmin.hour // BIN_HOURS) * BIN_HOURS)

# 2) Per-bin Kernel Density (1 km cell, 18 km radius) saved to the GDB
created = []
cur = start
while cur <= tmax:
    nxt = cur + dt.timedelta(hours=BIN_HOURS)
    where = (f"{arcpy.AddFieldDelimiters(PTS_METERS, TIME_FIELD)} >= TIMESTAMP '{cur:%Y-%m-%d %H:%M:%S}' "
             f"AND {arcpy.AddFieldDelimiters(PTS_METERS, TIME_FIELD)} < TIMESTAMP '{nxt:%Y-%m-%d %H:%M:%S}'")

    lyr_name = f"bin_{cur:%Y%m%d_%H%M}"
    
    arcpy.management.MakeFeatureLayer(PTS_METERS, lyr_name, where)
    # for f in arcpy.ListFields(lyr_name):
    #     if f.name == "population":
    #         print(f"Field '{f.name}': type={f.type}")
    # print(foo)
    try:
        if int(arcpy.management.GetCount(lyr_name).getOutput(0)) > 0:
            print(cur, tmax)
            out_name = f"{PREFIX}{cur:%Y%m%d_%H%M}"
            out_path = os.path.join(GDB, out_name)
            if arcpy.Exists(out_path):
                arcpy.management.Delete(out_path)

            kd = KernelDensity(
                in_features=lyr_name,
                population_field="population",
                cell_size=CELL_SIZE_M,
                search_radius=RADIUS_M,
                out_cell_values="DENSITIES",
                method="PLANAR"
            )
            kd.save(out_path)
            created.append(out_name)
    except Exception:
        raise RuntimeError("KernelDensity failed:\n" + arcpy.GetMessages(2))        
    cur = nxt
    arcpy.management.Delete(lyr_name)

if not created:
    raise RuntimeError("No rasters created. Check time field values and bin size.")

# 3) Create a time-aware Mosaic Dataset and add the rasters we just made
if arcpy.Exists(os.path.join(GDB, MD_NAME)):
    arcpy.management.Delete(os.path.join(GDB, MD_NAME))

MD = arcpy.management.CreateMosaicDataset(
    in_workspace=GDB,
    in_mosaicdataset_name=MD_NAME,
    coordinate_system=sr,
    pixel_type="32_BIT_FLOAT"
).getOutput(0)

# Add each slice explicitly (robust for FGDB rasters)
for nm in created:
    print(nm)
    arcpy.management.AddRastersToMosaicDataset(
        in_mosaic_dataset=MD,
        raster_type="Raster Dataset",
        input_path=os.path.join(GDB, nm),
        update_cellsize_ranges="UPDATE_CELL_SIZES",
        update_boundary="UPDATE_BOUNDARY",
        update_overviews="NO_OVERVIEWS"
    )

# 4) Add StartTime on the mosaic items and populate from item Name (kd_YYYYMMDD_HHMM)
if "StartTime" not in [f.name for f in arcpy.ListFields(MD)]:
    arcpy.management.AddField(MD, "StartTime", "DATE")

code_block = """import datetime
def parse_name(nm):
    # expects 'kd_YYYYMMDD_HHMM' 
    return datetime.datetime.strptime(nm[-13:], '%Y%m%d_%H%M')
"""
arcpy.management.CalculateField(MD, "StartTime", "parse_name(!Name!)", "PYTHON3", code_block)
# arcpy.management.EnableTime(MD, "StartTime", "Single", timeStepInterval=str(BIN_HOURS), timeStepUnits="HOURS")

# Build pyramids & stats (correct function name/case)
arcpy.management.BuildPyramidsandStatistics(MD, skip_existing="OVERWRITE")

print(f"OK: {len(created)} KDE slices → {MD}")


✅ time_bin_Converted already exists
2024-09-26 00:00:00 2024-09-27 16:00:00
2024-09-26 04:00:00 2024-09-27 16:00:00
2024-09-26 08:00:00 2024-09-27 16:00:00
2024-09-26 12:00:00 2024-09-27 16:00:00
2024-09-26 16:00:00 2024-09-27 16:00:00
2024-09-26 20:00:00 2024-09-27 16:00:00
2024-09-27 00:00:00 2024-09-27 16:00:00
2024-09-27 04:00:00 2024-09-27 16:00:00
2024-09-27 08:00:00 2024-09-27 16:00:00
2024-09-27 12:00:00 2024-09-27 16:00:00
2024-09-27 16:00:00 2024-09-27 16:00:00
kd_cities_cum_helene_pop_20240926_0000
kd_cities_cum_helene_pop_20240926_0400
kd_cities_cum_helene_pop_20240926_0800
kd_cities_cum_helene_pop_20240926_1200
kd_cities_cum_helene_pop_20240926_1600
kd_cities_cum_helene_pop_20240926_2000
kd_cities_cum_helene_pop_20240927_0000
kd_cities_cum_helene_pop_20240927_0400
kd_cities_cum_helene_pop_20240927_0800
kd_cities_cum_helene_pop_20240927_1200
kd_cities_cum_helene_pop_20240927_1600
OK: 11 KDE slices → C:\Users\colto\Documents\tw_project\tw_project\tw_project.gdb\KD_Time_citie